In [5]:
import re
import numpy as np
import argparse

import torch

In [ ]:
# utils.py


In [14]:
class RNNencoder(torch.nn.Module):
    def __init__(self):
        super(RNNencoder, self).__init__()
        
        
    def forward(self):
        return
    
    def init_hidden(self):
        return

In [ ]:
class Attention(torch.nn.Module):
    def __init__(self):
        super(Attention, self).__init__()
        
    def forward(self):
        
        return
    
    def score(self):
        
        return

In [13]:
class RNNdecoder(torch.nn.Module):
    def __init__(self):
        super(RNNdecoder, self).__init__()
        
    def forward(self):
        
        return

In [15]:
# seq2seq.py
class seq2seq(torch.nn.Module):
    '''
    Sequence to Sequence model implementation
    '''
    def __init__(self):
        super(seq2seq, self).__init__()
        
    def encode(self):
        
        return
    
    def decode(self):
        
        return
    
    def forward(self):
        
        return
        

In [16]:
model = seq2seq()